In [3]:
"""
Get all the match ids for a certain season
"""


import requests
from bs4 import BeautifulSoup
import os
import csv
from time import sleep

season = 1962 - 1954
year = season + 1954
print(year)
season_id = str(season) if season >= 10 else "0" + str(season)

URL = "https://www.voetbalstats.nl/listjaarere.php?seizoenid=" + season_id
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")

td_elements = soup.find_all("td", class_="linkere")

wedstrijden = []
for td in td_elements:
    a_tag = td.find("a", href=True)
    if a_tag and a_tag["href"].startswith("opstelere"):
        wedstrijden.append(a_tag["href"])

wedstrijden = list(set(wedstrijden))
print(len(wedstrijden))

ids = [int(x.split("=", 1)[1]) for x in wedstrijden]


1962
240


In [5]:
len(wedstrijden)

240